In [2]:
/**
 * Demonstrate how to solve a Second-Order Conic Programming (SOCP) problem using interior point method.
 * The initial guess is given by the SDPT3 algorithm.
 *
 * @author Haksun Li
 */

%use s2
import dev.nm.solver.IterativeSolution

// construct an SOCP problem to solve
fun problem() : SOCPGeneralProblem {
    // min f'x
    val f: Vector = DenseVector(1.0, 0.0, 0.0, 0.0, 0.0)

    // The A's in the conic constraints.
    val A1t: Matrix = DenseMatrix(arrayOf(doubleArrayOf(0.0, -1.0, 0.0, 1.0, 0.0), doubleArrayOf(0.0, 0.0, 1.0, 0.0, -1.0)))
    val A2t: Matrix = DenseMatrix(arrayOf(doubleArrayOf(0.0, 0.5, 0.0, 0.0, 0.0), doubleArrayOf(0.0, 0.0, 1.0, 0.0, 0.0)))
    val A3t: Matrix = DenseMatrix(arrayOf(doubleArrayOf(0.0, 0.0, 0.0, -0.7071, -0.7071), doubleArrayOf(0.0, 0.0, 0.0, -0.3536, 0.3536)))

    // The b's in the conic constraints.
    val b1: Vector = f
    val b2: Vector = f.ZERO()
    val b3: Vector = f.ZERO()

    // The c's in the conic constraints.
    val c1: Vector = DenseVector(2) // zero
    val c2: Vector = DenseVector(-0.5, 0.0)
    val c3: Vector = DenseVector(4.2426, -0.7071)

    // The d's in the conic constraints.
    val d = doubleArrayOf(0.0, 1.0, 1.0)
    val constraints: kotlin.collections.List<SOCPGeneralConstraint> = java.util.Arrays.asList<SOCPGeneralConstraint>(
        SOCPGeneralConstraint(A1t.t(), c1, b1, d[0]),
        SOCPGeneralConstraint(A2t.t(), c2, b2, d[1]),
        SOCPGeneralConstraint(A3t.t(), c3, b3, d[2])
    )

    // The SOCP problem to be solved.
    val problem: SOCPGeneralProblem = SOCPGeneralProblem(
        f, 
        constraints) // ||Ax + b|| <= c'x + d
    return problem
}

// Uses interior point method to solve the given problem from a given starting point.
val x0: Vector = DenseVector(1.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0)
val s0: Vector = DenseVector(3.7, 1.0, -3.5, 1.0, 0.25, 0.5, 1.0, -0.35355, -0.1767)
val y0: Vector = DenseVector(-3.7, -1.5, -0.5, -2.5, -4.0)
// an initial guess
val soln0 = PrimalDualSolution(x0, s0, y0)

// construct an SOCP solver
val solver = PrimalDualInteriorPointMinimizer(
    1e-8,
    20) // max number of iterations

// solve the SOCP problem
val p = problem()
val soln: IterativeSolution<PrimalDualSolution> = solver.solve(p)
soln.search() // Users do not provide initial guess.

// the solution
val y = soln.minimizer().y
val fx = p.f().evaluate(y);
println("minimizer: $y, fx = $fx")

minimizer: [-1.707790, -2.044705, -0.852730, -2.544836, -2.485646] , fx = -1.7077904474950314
